In [15]:
# Source: https://www.turing.com/kb/deploy-ml-models-with-flask-and-docker

# Data Manipulation libraries
import pandas as pd
from sklearn.model_selection  import train_test_split
from sklearn.neural_network import MLPRegressor
import joblib

In [28]:
# import data
df = pd.read_csv('data/MELBOURNE_HOUSE_PRICES_LESS.csv')

df_full = pd.read_csv('data/Melbourne_housing_FULL.csv')

# merge data
#df = pd.concat([df, df2], axis=0)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63023 entries, 0 to 63022
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         63023 non-null  object 
 1   Address        63023 non-null  object 
 2   Rooms          63023 non-null  int64  
 3   Type           63023 non-null  object 
 4   Price          48433 non-null  float64
 5   Method         63023 non-null  object 
 6   SellerG        63023 non-null  object 
 7   Date           63023 non-null  object 
 8   Postcode       63023 non-null  int64  
 9   Regionname     63023 non-null  object 
 10  Propertycount  63023 non-null  int64  
 11  Distance       63023 non-null  float64
 12  CouncilArea    63023 non-null  object 
dtypes: float64(2), int64(3), object(8)
memory usage: 6.3+ MB


In [29]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34857 entries, 0 to 34856
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Suburb         34857 non-null  object 
 1   Address        34857 non-null  object 
 2   Rooms          34857 non-null  int64  
 3   Type           34857 non-null  object 
 4   Price          27247 non-null  float64
 5   Method         34857 non-null  object 
 6   SellerG        34857 non-null  object 
 7   Date           34857 non-null  object 
 8   Distance       34856 non-null  float64
 9   Postcode       34856 non-null  float64
 10  Bedroom2       26640 non-null  float64
 11  Bathroom       26631 non-null  float64
 12  Car            26129 non-null  float64
 13  Landsize       23047 non-null  float64
 14  BuildingArea   13742 non-null  float64
 15  YearBuilt      15551 non-null  float64
 16  CouncilArea    34854 non-null  object 
 17  Lattitude      26881 non-null  float64
 18  Longti

In [17]:
df.head()

,Suburb,Address,Rooms,Type,Price,Method,SellerG,Date,Postcode,Regionname,Propertycount,Distance,CouncilArea
0,Abbotsford,49 Lithgow St,3,h,1490000.0,S,Jellis,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
1,Abbotsford,59A Turner St,3,h,1220000.0,S,Marshall,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
2,Abbotsford,119B Yarra St,3,h,1420000.0,S,Nelson,1/04/2017,3067,Northern Metropolitan,4019,3.0,Yarra City Council
3,Aberfeldie,68 Vida St,3,h,1515000.0,S,Barry,1/04/2017,3040,Western Metropolitan,1543,7.5,Moonee Valley City Council
4,Airport West,92 Clydesdale Rd,2,h,670000.0,S,Nelson,1/04/2017,3042,Western Metropolitan,3464,10.4,Moonee Valley City Council


In [18]:
df.describe()

,Rooms,Price,Postcode,Propertycount,Distance
count,63023.000000,4.843300e+04,63023.000000,63023.000000,63023.000000
mean,3.110595,9.978982e+05,3125.673897,7617.728131,12.684829
std,0.957551,5.934989e+05,125.626877,4424.423167,7.592015
min,1.000000,8.500000e+04,3000.000000,39.000000,0.000000
25%,3.000000,6.200000e+05,3056.000000,4380.000000,7.000000
50%,3.000000,8.300000e+05,3107.000000,6795.000000,11.400000
75%,4.000000,1.220000e+06,3163.000000,10412.000000,16.700000
max,31.000000,1.120000e+07,3980.000000,21650.000000,64.100000


In [19]:
# drop na prices
df = df[df['Price'].notna()]

# get dummie for type
df = pd.get_dummies(df, columns=['Type'])

drop_cols = ['Suburb',
             'Address',
             'Method',
             'SellerG',
             'Date',
             'Postcode',
             'Regionname',
             'Propertycount',
             'CouncilArea',
             'Type_u'
]

df.drop(drop_cols, axis=1, inplace=True)



df.head()

,Rooms,Price,Distance,Type_h,Type_t
0,3,1490000.0,3.0,True,False
1,3,1220000.0,3.0,True,False
2,3,1420000.0,3.0,True,False
3,3,1515000.0,7.5,True,False
4,2,670000.0,10.4,True,False


In [20]:
df = df[df['Rooms']<9]



df.describe()

,Rooms,Price,Distance
count,48420.000000,4.842000e+04,48420.000000
mean,3.069186,9.974567e+05,12.702912
std,0.927984,5.922407e+05,7.550062
min,1.000000,8.500000e+04,0.000000
25%,2.000000,6.200000e+05,7.000000
50%,3.000000,8.300000e+05,11.700000
75%,4.000000,1.220000e+06,16.700000
max,8.000000,1.120000e+07,55.800000


In [21]:
X = df[['Rooms', 'Distance', 'Type_h', 'Type_t']]
y = df[['Price']]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 108)

# save the scaler
joblib.dump(scaler, "model/scaler.pkl")

['model/scaler.pkl']

In [22]:
X_train

,Rooms,Distance,Type_h,Type_t
34016,2.080676,-0.397738,0.646487,-0.338587
25815,-2.229788,-0.530188,-1.546822,-0.338587
29704,-0.074556,-0.834825,0.646487,-0.338587
19156,-0.074556,4.396975,0.646487,-0.338587
9439,-1.152172,-0.305022,0.646487,-0.338587
...,...,...,...,...
41554,1.003060,0.145310,0.646487,-0.338587
46446,2.080676,-1.112971,0.646487,-0.338587
43467,-1.152172,1.893658,-1.546822,-0.338587
41105,-1.152172,-0.795089,-1.546822,-0.338587


In [23]:
from sklearn.ensemble import RandomForestRegressor
regressor =  RandomForestRegressor() 
regressor.fit(X_train, y_train.values.ravel())

#predicting the test set result
y_pred = regressor.predict(X_test)#put results as a DataFrame

# RSME
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(y_test, y_pred))
print(rmse)

356802.0091077383


In [24]:
#Saving the machine learning model to a file
joblib.dump(regressor, "model/price_model.pkl")

['model/price_model.pkl']

In [25]:
X_test

,Rooms,Distance,Type_h,Type_t
36423,-0.074556,-1.218931,0.646487,-0.338587
41662,-0.074556,1.045974,0.646487,-0.338587
13335,1.003060,0.449946,0.646487,-0.338587
12115,-1.152172,0.145310,-1.546822,-0.338587
18804,1.003060,-0.040121,-1.546822,-0.338587
...,...,...,...,...
31619,1.003060,-0.252042,0.646487,-0.338587
39319,-1.152172,-0.199062,-1.546822,2.953454
43249,1.003060,0.701603,0.646487,-0.338587
13712,-0.074556,0.330741,0.646487,-0.338587


In [26]:
y_pred

array([1651234.22820766,  493147.94017466, 1000495.48966175, ...,
        798320.80007768, 1578832.69196736,  829284.19244561])

In [27]:
import json

data = {"Rooms":"3","Distance":"45","Type_h":1,"Type_t":0}

js = json.dumps(data)

df_json = pd.DataFrame(data, index=[0])

scaler = joblib.load('model/scaler.pkl')

x_scaled = scaler.transform(df_json)

x_scaled = pd.DataFrame(x_scaled, columns=df_json.columns)

x_scaled

,Rooms,Distance,Type_h,Type_t
0,-0.074556,4.277769,0.646487,-0.338587
